<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Preliminaries" data-toc-modified-id="Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preliminaries</a></span><ul class="toc-item"><li><span><a href="#Data_SRC" data-toc-modified-id="Data_SRC-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data_SRC</a></span></li><li><span><a href="#Load-libraries" data-toc-modified-id="Load-libraries-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load libraries</a></span></li></ul></li><li><span><a href="#Demo-of-process_mpd" data-toc-modified-id="Demo-of-process_mpd-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Demo of process_mpd</a></span><ul class="toc-item"><li><span><a href="#counting-the-number-of-playlist" data-toc-modified-id="counting-the-number-of-playlist-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>counting the number of playlist</a></span></li><li><span><a href="#get-unique-tracks-in-all-playlist" data-toc-modified-id="get-unique-tracks-in-all-playlist-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>get unique tracks in all playlist</a></span></li><li><span><a href="#Iterate-on-files-and-playlist-level" data-toc-modified-id="Iterate-on-files-and-playlist-level-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Iterate on files and playlist level</a></span></li><li><span><a href="#create-random-subset-of-1%-of-all-playlists" data-toc-modified-id="create-random-subset-of-1%-of-all-playlists-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>create random subset of 1% of all playlists</a></span></li></ul></li><li><span><a href="#Make-dictionaries" data-toc-modified-id="Make-dictionaries-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Make dictionaries</a></span><ul class="toc-item"><li><span><a href="#Entire-dataset" data-toc-modified-id="Entire-dataset-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Entire dataset</a></span></li><li><span><a href="#subset-5k" data-toc-modified-id="subset-5k-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>subset 5k</a></span></li></ul></li><li><span><a href="#Get-audio-features" data-toc-modified-id="Get-audio-features-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get audio features</a></span><ul class="toc-item"><li><span><a href="#setup" data-toc-modified-id="setup-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>setup</a></span></li><li><span><a href="#Read-tracks" data-toc-modified-id="Read-tracks-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Read tracks</a></span><ul class="toc-item"><li><span><a href="#subset" data-toc-modified-id="subset-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>subset</a></span></li></ul></li><li><span><a href="#spotify-web-api" data-toc-modified-id="spotify-web-api-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>spotify-web api</a></span><ul class="toc-item"><li><span><a href="#Audio-features" data-toc-modified-id="Audio-features-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Audio features</a></span></li><li><span><a href="#Popularity" data-toc-modified-id="Popularity-4.3.2"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>Popularity</a></span></li><li><span><a href="#Genres" data-toc-modified-id="Genres-4.3.3"><span class="toc-item-num">4.3.3&nbsp;&nbsp;</span>Genres</a></span></li></ul></li></ul></li></ul></div>

# Preliminaries
## Data_SRC

In [1]:
#location of the dataset
#DATA_SRC='/home/beangoben/ml_data/mpd.v1/data'
DATA_SRC='/home/beangoben/data/mpd.v1/data'
#DATA_SRC='../data/raw/5k_subset'
# add src scripts to utilize
import sys
sys.path.append('../src')
sys.path.append('../src/data/')
sys.path.append('../src/models/')
sys.path.append('../src/features/')
sys.path.append('../src/visualization/')
# will reload any library
%load_ext autoreload
%autoreload 2

## Load libraries

In [2]:
import os
import json
import compressed_pickle as cpick
from collections import defaultdict, OrderedDict
# utility
from tqdm import tqdm_notebook as tqdm
# scientific python
import numpy as np
import pandas as pd
# nice printing
from pprint import pprint
# for plotting
import matplotlib.pyplot as plt
import seaborn as sns
from plot_utils import html_header, glance_dict
%matplotlib inline

# Demo of process_mpd

Located in data/data_utils.py, **process_mpd** will take a directory path, a function to operate on a playlist and a object to store results (could be a dataframe or a dict for example). 

Optional is a *max_n* (max number of slices to operate on) and use *rand=False* for random or sorted files.


## counting the number of playlist

In [3]:
from data_utils import process_mpd

def count_playlist(playlist,results):
    results['count']=results['count']+1
    return

In [4]:
results={}
results['count']=0
process_mpd(DATA_SRC,count_playlist,results,max_n=10,rand=True)
pprint(results)


{'count': 10000}


## get unique tracks in all playlist

In [5]:
from data_utils import process_mpd, normalize_name

def unique_trackids(playlist,results):
    pid = playlist['pid']
    
    rows=[]
    for track in playlist['tracks']:
        rows.append({'title':track['track_name'],'track_uri':track['track_uri'],'artist':track['artist_name']
                        ,'artist_uri':track['artist_uri']})
    results['count']+=1
    results['count']+=1
    pd.DataFrame(results).to_csv('test.csv',index=False,compression='gzip')
    return

In [6]:
results={}
results['count']=0
results['rows']
process_mpd(DATA_SRC,unique_trackids,results)
print(len(results))



334487


In [16]:
df = pd.DataFrame(results)
print(df.shape)
df = df.drop_duplicates('uri')
print(df.shape)
df.to_csv('../data/interim/5k_track_uri.csv',index=False,compression='gzip')

(335094, 4)
(107781, 4)


## Iterate on files and playlist level

In [7]:
from data_utils import process_mpd2, normalize_name

def pl_func(playlist,results):
    pid = playlist['pid']
    tracks=[turi2tint[track['track_uri'].replace('spotify:track:','')] for track in playlist['tracks']]
    results[pid]=tracks
    return

def json_func(indx,results_json,results_pl):
    if indx%100==0:
        cpick.save('playlist2track_ints',results_pl)
    return

In [14]:
from collections import OrderedDict
results = [None]*1000000
process_mpd2(DATA_SRC, pl_func,results, json_func, [])

KeyError: '76XXefqNyRrP3pm3uUckWS'

In [19]:
results = cpick.load('../interim/track_uri2track_name.pkl.bz2')

File ../interim/track_uri2track_name.pkl.bz2 cannot be read


AttributeError: 'FileNotFoundError' object has no attribute 'decode'

In [16]:
turi2tint['76XXefqNyRrP3pm3uUckWS']

KeyError: '76XXefqNyRrP3pm3uUckWS'

In [17]:
import compressed_pickle as cpick

cpick.save('test',tracks)

In [19]:
filenames = os.listdir('.')
df = pd.DataFrame()
tracks={}
def is_dat(x): return x.startswith("test_") and x.endswith(".csv")
good_files = [f for f in filenames if is_dat(f)]
for filename in tqdm(good_files):
    a_df = pd.read_csv(filename,compression='bz2')
    df
    

## create random subset of 1% of all playlists

In [37]:
from data_utils import process_mpd
import random

def coinflip(percent=0.001):
    return random.uniform(0, 1) < percent

def random_subset(playlist,results):
    if coinflip(1):
        results.append(playlist)
    return

In [ ]:
results=[]
process_mpd(DATA_SRC,random_subset,results)
print(len(results))
pprint(results[-1])

 86%|████████▋ | 863/1000 [06:15<00:59,  2.30it/s]

In [27]:
a_dict={"info": {
        "generated_on": "2017-12-03 08:41:42.057563", 
        "slice": "random subset of 0.1% of playlist", 
        "version": "v1"
    }, "playlists": results}
with open('../data/intetim/random_01_percent.json','w') as afile:
    afile.write(json.dumps(a_dict, indent=4))

# Make dictionaries
## Entire dataset

In [ ]:
from data_utils import process_mpd, normalize_name

# track uri vs track name
def track_uri2track_name(playlist,results):
    for track in playlist['tracks']:
        track_uri=track['track_uri'].replace('spotify:track:','')
        results[track_uri]=track['track_name']
    return
html_header('track_uri2track_name')
results={}
process_mpd(DATA_SRC,track_uri2track_name,results)
cpick.save('../data/interim/track_uri2track_name',results)
# track integer vs track uri
html_header('track_int2track_uri')
tint2turi = {indx:item for indx,item in enumerate(results)}
cpick.save('../data/interim/track_int2track_uri.pkl.bz2',tint2turi)
html_header('track_uri2track_int')
turi2tint = {item:indx for indx,item in enumerate(results.keys())}
cpick.save('../data/interim/track_uri2track_int.pkl.bz2',turi2tint)
# track uri vs artist
def track_uri2artist_uri(playlist,results):
    for track in playlist['tracks']:
        track_uri=track['track_uri'].replace('spotify:track:','')
        artist_uri=track['artist_uri'].replace('spotify:artist:','')
        results[track_uri]=artist_uri
    return
html_header('track_uri2artist_uri')
results={}
process_mpd(DATA_SRC,track_uri2artist_uri,results)
cpick.save('../data/interim/track_uri2artist_uri',results)
# artist uri vs artist name
def artist_uri2artist_name(playlist,results):
    for track in playlist['tracks']:
        artist_uri=track['artist_uri'].replace('spotify:artist:','')
        results[artist_uri]=track["artist_name"]
    return
html_header('artist_uri2artist_name')
results={}
process_mpd(DATA_SRC,artist_uri2artist_name,results)
cpick.save('../data/interim/artist_uri2artist_name',results)
html_header('artist_name2artist_uri (list)')
aname2auri=defaultdict(list)
for auri,aname in results.items():
    aname2auri[aname].append(auri)
cpick.save('../data/interim/artist_name2artist_uri',aname2auri)
# artist_uri2track_uri
def artist_uri2track_uri(playlist,results):
    for track in playlist['tracks']:
        track_uri=track['track_uri'].replace('spotify:track:','')
        artist_uri=track['artist_uri'].replace('spotify:artist:','')
        results[artist_uri].add(track_uri)
    return
html_header('artist_uri2track_uri')
results=defaultdict(set)
process_mpd(DATA_SRC,artist_uri2track_uri,results)
cpick.save('../data/interim/artist_uri2track_uri',{key:list(val) for key,val in results.items()})
# track uri 2 album uri
def track_uri2album_uri(playlist,results):
    for track in playlist['tracks']:
        track_uri=track['track_uri'].replace('spotify:track:','')
        album_uri=track['album_uri'].replace('spotify:album:','')
        results[track_uri]=album_uri
    return
html_header('track_uri2album_uri')
results={}
process_mpd(DATA_SRC,track_uri2album_uri,results)
cpick.save('../data/interim/track_uri2album_uri',results)
# playlist and track_ints
def playlist2track_ints(playlist,results):
    pid = playlist['pid']
    tracks=[turi2tint[track['track_uri'].replace('spotify:track:','')] for track in playlist['tracks']]
    results[pid]=tracks
    return
html_header('playlist2track_ints')
results = [None]*1000000
process_mpd(DATA_SRC,playlist2track_ints,results)
cpick.save('../data/interim/playlist2track_ints',results)

## subset 5k

In [ ]:
def make_subset(subset,original,prefix='5k-'):
    adir = '/'.join(original.split('/')[:-1])
    name = original.split('/')[-1]
    data = cpick.load(original)
    sub = {i:data[i] for i in subset}
    new_file = os.path.join(adir,prefix+name)
    cpick.save(new_file,sub)
    return

In [ ]:
html_header('playlist')
pl2tint = cpick.load('../data/interim/playlist2track_ints.pkl.bz2')[:5000]
cpick.save('../data/interim/5k-playlist2track_ints',pl2tint)
sub_tint = list(set([t for pl in pl2tint for t in pl]))
make_subset(sub_tint,'../data/interim/track_int2track_uri.pkl.bz2')

# Get audio features
## setup

In [3]:
import spotify
import pandas as pd

## Read tracks

In [4]:
turi2tname = cpick.load('../data/interim/track_uri2track_name.pkl.bz2')
turi2auri = cpick.load('../data/interim/track_uri2artist_uri.pkl.bz2')
print(len(turi2tname))
df = pd.DataFrame(list(turi2tname.keys()),columns=['track_uri'])
df['track_name'] = df['track_uri'].apply(lambda x: turi2tname[x])
df['artist_uri'] = df['track_uri'].apply(lambda x: turi2auri[x])
print(df.shape)
df.head()

2262292
(2262292, 3)


,track_uri,track_name,artist_uri
0,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),2wIVse2owClT7go1WT98tk
1,6I9VzXrHxO9rA9A5euc8Ak,Toxic,26dSoYclwsYLMAKD3tpOr4
2,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,6vWDO969PvNqNYHIOW5v0m
3,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,31TPClRtHm23RisEBtV3X7
4,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,5EvFsr3kj42KNv97ZEnqij


### subset

In [29]:
pl2tint = cpick.load('../data/interim/5k_playlist2track_ints.pkl.bz2')
tint2turi = cpick.load('../data/interim/track_int2track_uri.pkl.bz2')
sub_turi = list(set([tint2turi[t] for pl in pl2tint for t in pl]))
df = df[df['track_uri'].isin(sub_turi)]
print(df.shape)
df.head()

KeyboardInterrupt: 

## spotify-web api

### Audio features

In [29]:
interest=['acousticness','danceability','duration_ms','energy',
          'liveness','loudness','speechiness','tempo','valence']
def get_audio_features(indx,chunk,sp,results):
    tids = chunk['track_uri'].tolist()
    afeatures = sp.audio_features(tids)
    spotify.replace_none(afeatures)
    a_df = pd.DataFrame(afeatures)[interest]
    a_df['track_uri']=tids
    results = results.append(a_df,ignore_index=True)
    if indx%1000 == 0:
        cpick.save('audio_features',results)
        
    return results

In [40]:
results = pd.DataFrame()
results = spotify.df_chunk_apply(df, 100, get_audio_features,results,start=1700100)
results = results.set_index('track_uri')
cpick.save('audio_features',results)
print(results.shape)
results.head(1)

HBox(children=(IntProgress(value=0, max=5622), HTML(value='')))

Saving to audio_features.pkl.bz2



Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/beangoben/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/beangoben/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/beangoben/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



Saving to audio_features.pkl.bz2
retrying ...1secs
Saving to audio_features.pkl.bz2
Saving to audio_features.pkl.bz2
Saving to audio_features.pkl.bz2
Saving to audio_features.pkl.bz2

Saving to audio_features.pkl.bz2
(2262592, 10)


,acousticness,danceability,duration_ms,energy,liveness,loudness,speechiness,tempo,valence,track_uri
0,0.0311,0.904,226864.0,0.813,0.0471,-7.105,0.121,125.461,0.81,0UaMYEvWZi0ZqiDOoHU3YI


### Popularity

In [ ]:
def get_popularity(indx,chunk,sp,results):
    tids = chunk['track_uri'].tolist()
    tracks = sp.tracks(tids)
    data = tracks['tracks']
    spotify.replace_none(data)  
    #pop =[{'popularity':t['popularity'],'track_uri':t['id']} if t is not None else {'popularity':np.nan,'track_uri':t['id']} for t in tracks['tracks']]
    a_df = pd.DataFrame(data)[['popularity']]
    a_df['track_uri']=tids
    results = results.append(a_df,ignore_index=True)
    if indx%1000 == 0:
        cpick.save('5k-popularity',results)
    return results

In [ ]:
#results = pd.DataFrame()
results = spotify.df_chunk_apply(df, 50, get_popularity,results)
results = results.set_index('track_uri')
cpick.save('5k-popularity',results)
print(results.shape)
results.head(1)

### Genres

In [18]:
def get_artist_genres(indx,chunk,sp,results):
    auris = chunk['artist_uri'].apply(lambda x: x).tolist()
    arts = sp.artists(auris)
    data = arts['artists']
    for auri,datum in zip(auris,data):
        if datum is None:
            results[auri]=[]
        else:
            results[auri]=datum['genres']

    if indx%1000 == 0:
        cpick.save('genres',results)
    return results

In [22]:
#results = {}
results = spotify.df_chunk_apply(df, 50, get_artist_genres,results,start=221400)
cpick.save('genres',results)
print(results.shape)
results.head(1)

HBox(children=(IntProgress(value=0, max=40818), HTML(value='')))

Saving to genres.pkl.bz2



Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/beangoben/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/beangoben/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/beangoben/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2
Saving to genres.pkl.bz2


AttributeError: 'dict' object has no attribute 'shape'

In [ ]:
def get_artist(indx,chunk,sp):
    tids = chunk['uri'].apply(lambda x: x.replace('spotify:track:','')).tolist()
    tracks = sp.tracks(tids)    
    arts =[{'artist_uri':t['artists'][0]['uri']} if t is not None else {'artist_uri':None}  for t in tracks['tracks']]
    return arts


In [132]:
display(af_df.loc[a_df[pd.isnull(a_df['artist_uri'])].index]['track_href'])
display(df.loc[a_df[pd.isnull(a_df['artist_uri'])].index])

20622     https://api.spotify.com/v1/tracks/656TZlNdVe90...
28830     https://api.spotify.com/v1/tracks/5GiU7GOYjDH2...
36627     https://api.spotify.com/v1/tracks/2ZF3gpgGPjOf...
45507     https://api.spotify.com/v1/tracks/2odGdanrUsgu...
64596     https://api.spotify.com/v1/tracks/6tf7ENqpF89X...
78710     https://api.spotify.com/v1/tracks/0fJ6UuZ7onqc...
79513     https://api.spotify.com/v1/tracks/0Jb7g80rInlj...
95918     https://api.spotify.com/v1/tracks/1HubPEsZLy8R...
102653    https://api.spotify.com/v1/tracks/6CNzVurjdLR4...
Name: track_href, dtype: object

,artist,pid,title,uri,csv_id,artist_uri
20622,son palenque,497,palenque palenque,spotify:track:656TZlNdVe90zHvmebFt9U,20622,NaN
28830,coldplay,782,adventure of a lifetime,spotify:track:5GiU7GOYjDH2yp7fMf9w9j,28830,NaN
36627,bing crosby,1065,white christmas,spotify:track:2ZF3gpgGPjOfi1yTWGX6Bm,36627,NaN
45507,dionne warwick,1425,walk on by - live,spotify:track:2odGdanrUsgu0d2i2Q5XoT,45507,NaN
64596,thomas newson,2331,wildfire - radio edit,spotify:track:6tf7ENqpF89Xox3xyNPGMv,64596,NaN
78710,alex aiono,3114,i hate you i love,spotify:track:0fJ6UuZ7onqcldGr8HT5QV,78710,NaN
79513,marvin gaye,3158,third world girl,spotify:track:0Jb7g80rInlj3fa6W65Bp2,79513,NaN
95918,mat joe,4140,make a living,spotify:track:1HubPEsZLy8R1HuCzMNBCd,95918,NaN
102653,jake spooner,4567,rivals,spotify:track:6CNzVurjdLR4seIfAsRxai,102653,NaN


In [219]:


df = fix_artist_uri(df,pd.isnull(df['artist_uri']),sp)

Fixed 0 out of 0


In [112]:
#df.merge(a_df[['artist_uri']],left_index=True,right_index=True).to_csv('../data/interim/5k_track_uri.csv',index=False)